In [1]:
import json
import pickle
from functools import reduce
from collections import defaultdict

In [8]:
class entity:
    def __init__(self, recipe):
        self.recipe = recipe
        self.ingredient = [x for x in recipe["food_ingredients"]]
        self.number = [0.0 for x in recipe["food_ingredients"]]
        self.unit = ["TBD" for x in recipe["food_ingredients"]]
        self.entities = ["TBD" for x in recipe["food_ingredients"]]
#         self.failure
#         self.recipe_triple = []
    def get_nlpres(self, nlp_output, index):
        if not nlp_output:
            self.number[index]="TBD"
            self.unit[index] = "TBD"
            self.entities[index] = "TBD"
#             print("")
            return
        res = 0.0
        for v in nlp_output:
            if v[1] != 'CARDINAL' and v[1] != "QUANTITY":
#                 print(v)
                pass
            elif v[1] == "CARDINAL":
                vals = v[0].split()
#                 res = 0.0
                for _ in vals:
                    try:
                        res+=eval(_)
                    except:
                        break
                self.number[index]=res
        
#                 self.unit.append("TBD")
#                 self.entities.append("TBD")
            else:
                vals = v[0].split()
#                 res = 0.0
                for _ in vals:
                    try:
                        res += eval(_)
                    except:
                        self.unit[index] = _
                        break
                self.number[index] = res
#                 self.entities.append("TBD")
            #TBD as default since cannot find

In [84]:
recipes[0].unit

['cup', 'cup', 'teaspoon', 'cup', 'teaspoon', 'cup', 'cup', 'DEFAULT']

In [85]:
recipes = pickle.load(open("./third_round_extraction.pkl", "rb"))

In [86]:
dir(recipes[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'entities',
 'get_nlpres',
 'ingredient',
 'number',
 'recipe',
 'unit']

In [87]:
all_units = reduce(lambda x,y:x.union(y) ,(map(lambda x: set(x.unit), recipes)))

In [88]:
len(all_units)

14

## Solving units
### First Round Res:

"4x4-inch", "1-inch", "1/2-inch", "1/8-inch", "12-inch", "2-inch", "degree", "f/45", "fluid", "12x12", "half"  -> remove inches out,in nlp again

"caramel", "chicken", "cloves","garlic", "graham", "ice", "maraschino", "pint", "portobello", "whole", "(" -> DEFAULT

"cube", 'cup', "gallon", "gram","liter", "milliliter", "oz", "pound", "quart", "tablespoon", "teaspoon"

"ounce" to "oz"

"not valid", "red", "yellow" Remove

"skinless" -> in entity, change it to "chicken breast halves", in unit, change to default and add entities as "chicken breast halves"

In [89]:
all_foods = reduce(lambda x,y:x.union(y) ,(map(lambda x: set(x.entities), recipes)))

In [90]:
# statistic:
lenthoffoods = defaultdict(set)
for a in all_foods:
    lenthoffoods[len(a.split())].add(a)

In [91]:
lenthoffoods.keys()

dict_keys([0, 4, 5, 3, 2, 6, 7, 1, 8, 10, 9, 11])

In [92]:
len(lenthoffoods[7])

53

In [93]:
lenthoffoods[11]

{'package seasoned crouton such new brand texas toast sea salt pepper'}

In [57]:
lenthoffoods[10]

{'fat free creamy salad dressing fat free miracle whip tm',
 'g tub philadelphia savoury garlic cooking creme cream cheese product',
 'garlic flavor bite size bagel chip regular size bagel chip'}

In [71]:
lenthoffoods[0]

{''}

In [70]:
lenthoffoods[7]

{'bag eye recipe ready pizza supreme blend',
 'bag white cheddar kettle style potato chip',
 'box chicken flavor dry bread stuffing mix',
 'box dry corn muffin mix such jiffy',
 'brandy base orange liqueur such grand marnier',
 'citrus flavor energy drink red bull ™',
 'cup blue green fruit flavor candy such',
 'dash hot pepper sauce such frank redhot',
 'dash hot pepper sauce texas pete ™',
 'drain hot mild pickle banana pepper ring',
 'envelope fleischmann pizza crust yeast rapidrise yeast',
 'fluid ounce gran gala triple orange liqueur',
 'fluid ounce triple sec orange flavor liqueur',
 'foil wrap milk chocolate piece such hershey',
 'frozen bite size potato nugget such tater',
 'g package naan bread bread g package',
 'jar real bacon piece such hormel ™',
 'light rye flour such bob red mill',
 'low fat greek style yogurt such fage',
 'ounce bottle buffalo wing sauce such frank',
 'ounce cream mushroom soup regular % sodium',
 'ounce envelope fleischmann pizza crust yeast yeast',
 '

### Solving entities
1. length 0,7,8,9,10,11 remove


In [94]:
len(recipes)

3828

In [95]:
list(lenthoffoods[2])[5]

'lemon peel'

In [96]:
from copy import copy
reduced_food = copy(lenthoffoods)
rm_list = [0,7,8,9,10,11]
for r in rm_list:
    del reduced_food[r]

In [97]:
"ounce" in lenthoffoods[1]

True

### TODO
1. Remove units word from all list
2. Only keeps Noun in 1-length list
3. TOBECONTINUED

In [97]:
import spacy
import regex
import json
from collections import Counter
nlp =  spacy.load('en_core_web_lg')
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
from spacy.tokenizer import Tokenizer
Tokenizer = Tokenizer(nlp.vocab)
Lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

In [98]:
all_units.add("ounce")
bad_words = set(list(all_units)+list({"1/4-inch", "4x4-inch", "1-inch", "1/2-inch", "1/8-inch", "12-inch", "2-inch", "degree", "f/45", "fluid", "12x12", "half", "degree degree"}))

In [99]:
bad_words

{'1-inch',
 '1/2-inch',
 '1/4-inch',
 '1/8-inch',
 '12-inch',
 '12x12',
 '2-inch',
 '4x4-inch',
 'DEFAULT',
 'cube',
 'cup',
 'degree',
 'degree degree',
 'f/45',
 'fluid',
 'gallon',
 'gram',
 'half',
 'inch',
 'liter',
 'milliliter',
 'not valid',
 'ounce',
 'oz',
 'pound',
 'quart',
 'tablespoon',
 'teaspoon'}

In [100]:
tmp = copy(reduced_food[1])
len(tmp)

219

In [101]:
for t in tmp:
    if t in bad_words:
        reduced_food[1].remove(t)

In [150]:
def writeres(length, l):
    file = "./"+str(length)+".txt"
    with open(file, "w") as f:
        for v in l:
            f.write(v)
            f.write("\n")
    


In [102]:
out_1 = []
for r in reduced_food[1]:
    tmp=nlp(r)
    for token in tmp:
        if token.tag_ == "NN":
            out_1.append(token.text)
len(out_1)
            

182

In [101]:
writeres(1,out_1)

In [103]:
# finished lenth 1 items, begin with more than 2
len(reduced_food[2])

1171

In [104]:
outputs = set()
for key, value in reduced_food.items():
    if key != 1:
        tmp = copy(value)
        for t in tmp:
            res = []
            for v in t.split():
                if v not in bad_words:
                    res.append(v)
            if res:
                outputs.add(" ".join(res))
                

In [111]:
outputs = set(list(outputs)+list(out_1))

In [112]:
len(outputs)

3973

In [107]:
# add to 1-itmset
outputs_dict = defaultdict(set)
for o in outputs:
    outputs_dict[len(o.split())].add(o)

In [110]:
outputs_dict.keys()

dict_keys([2, 3, 1, 4, 5, 6])

In [165]:
for k, v in outputs_dict.items():
    writeres(k, v)

In [113]:
len(outputs_dict[2])

1178

In [114]:
len(outputs_dict[3])

1450

In [115]:
len(outputs_dict[4])

753

In [116]:
len(outputs_dict[5])

300

In [117]:
len(outputs_dict[6])

80

In [166]:
writeres("unit", all_units)

### processing 3rd round recipe

In [189]:
recipes = pickle.load(open("./third_round_extraction.pkl", "rb"))

In [190]:
# 0,7,8,9,10,11 remove
rmlist = [0,7,8,9,10,11]
for r in recipes:
    for index in range(len(r.unit)):
        if len(r.entities[index].split()) in rmlist:
            r.ingredient[index] = "not valid"
            r.unit[index] = "not valid"
            r.number[index] = "not valid"
            r.entities[index] = "not valid"

In [191]:
all_len = set()
for r in recipes:
    for index in range(len(r.unit)):
        all_len.add(len(r.entities[index].split()))
all_len

{1, 2, 3, 4, 5, 6}

In [192]:
for r in recipes:
    for index in range(len(r.unit)):
        if len(r.entities[index].split()) in rmlist:
            print("???")

In [150]:
import spacy
import regex
import json
from collections import Counter
nlp =  spacy.load('en_core_web_lg')
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
from spacy.tokenizer import Tokenizer
Tokenizer = Tokenizer(nlp.vocab)
Lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)

In [196]:
# func1

check_1 = set()
for r in recipes:
    for index in range(len(r.unit)):
        if len(r.entities[index].split()) == 1:
            tmp = nlp(r.entities[index])
            for token in tmp:
                if token.tag_ == 'NN' and token.text not in bad_words:
                    r.entities[index] = token.text
                    check_1.add(token.text)
                else:
                    r.unit[index] = "not valid"
                    r.number[index] = "not valid"
                    r.ingredient[index] = "not valid"
                    r.entities[index] = "not valid"

In [197]:
len(check_1)

203

In [195]:
# func2
import copy
for r in recipes:
    for index in range(len(r.unit)):
        if len(r.entities[index].split()) > 1:
            tmp = copy.copy(r.entities[index].split())
            res = []
            for v in tmp:
                if v not in bad_words:
                    res.append(v)
            if res:
                r.entities[index] = " ".join(res)

In [205]:
## func1-> func2-> func1

In [210]:
lenset = set()
checkset = defaultdict(set)
for r in recipes:
    for index in range(len(r.unit)):
        lenset.add(len(r.entities[index].split()))
        checkset[len(r.entities[index].split())].add(r.entities[index])

In [166]:
one  = set()
with open("./1.txt", "r") as f:
    for line in f:
        one.add(line.strip())
two = set()
with open("./2.txt", "r") as f:
    for line in f:
        two.add(line.strip())
three = set()
with open("./3.txt", "r") as f:
    for line in f:
        three.add(line.strip())
four = set()
with open("./4.txt", "r") as f:
    for line in f:
        four.add(line.strip())
five = set()
with open("./5.txt", "r") as f:
    for line in f:
        five.add(line.strip())
six = set()
with open("./6.txt", "r") as f:
    for line in f:
        six.add(line.strip())

In [211]:
checkset[1].difference(one)

set()

In [212]:
checkset[2].difference(two)

set()

In [213]:
checkset[3].difference(three)

set()

In [214]:
checkset[4].difference(four)

set()

In [215]:
checkset[5].difference(five)

set()

In [216]:
checkset[6].difference(six)

set()

In [207]:
final_remove_set = {'half half', 'ounce half', 'fluid ounce half half'} # final_remove set for removing unsolved

In [208]:
cnt = 0
for r in recipes:
    for index in range(len(r.unit)):
        if r.entities[index] in final_remove_set:
            r.entities[index] = "not valid"
            r.unit[index] = "not valid"
            r.ingredient[index] = "not valid"
            r.number[index] = "not valid"
            cnt +=1

In [209]:
cnt

37

In [161]:
len(checkset[2])

1180

In [162]:
len(checkset[3])

1450

In [163]:
len(checkset[4])

754

In [164]:
len(checkset[5])

300

In [165]:
len(checkset[6])

80

In [79]:
checkset[6]

{'baked whole wheat regular pizza crust',
 'box vanilla wafer cookie such nilla',
 'candy coat milk chocolate piece such',
 'cayenne pepper sauce such frank redhot',
 'chop sun dry tomato olive oil',
 'clear plastic latex free disposable glove',
 'creamy salad dressing such miracle whip',
 'creole seasoning blend such link footnote',
 'crush chocolate sandwich cookie such oreo',
 'dash crush red pepper flake optional',
 'dash hot pepper sauce such tabasco',
 'dash hot sauce such frank redhot',
 'dash hot sauce such tabasco optional',
 'dash pepper sauce such red hot',
 'envelope reduce sodium taco seasoning mix',
 'herb season dry bread stuffing mix',
 'high protein crisp rice wheat cereal',
 'hot chile paste such sambal oelek',
 'hot pepper sauce such frank redhot',
 'jar chocolate hazelnut spread such nutella',
 'large leek white pale green part',
 'loaf frozen whole wheat bread dough',
 'loaf process cheese food such velveeta',
 'mexican style corn red green pepper',
 'milk chocolat

In [17]:
recipes[0].entities

['butter',
 'white sugar',
 'vanilla extract',
 'purpose flour',
 'salt',
 'unsweetened cocoa powder',
 'chop pecan',
 'cup confectioner sugar decoration']

In [52]:
bad_words

{'1-inch',
 '1/2-inch',
 '1/4-inch',
 '1/8-inch',
 '12-inch',
 '12x12',
 '2-inch',
 '4x4-inch',
 'DEFAULT',
 'cube',
 'cup',
 'degree',
 'degree degree',
 'f/45',
 'fluid',
 'gallon',
 'gram',
 'half',
 'inch',
 'liter',
 'milliliter',
 'not valid',
 'ounce',
 'oz',
 'pound',
 'quart',
 'tablespoon',
 'teaspoon'}

In [180]:
recipes[7].entities

['pie crust',
 'white sugar',
 'purpose flour',
 'cornstarch',
 'salt',
 'milk',
 'egg yolk',
 'square unsweetened chocolate',
 'butter',
 'vanilla extract']

In [217]:
file = "./final_round_extraction.pkl"
with open(file, "wb") as w:
    pickle.dump(recipes, w)

In [206]:
all_units

{'DEFAULT',
 'cube',
 'cup',
 'gallon',
 'gram',
 'inch',
 'liter',
 'milliliter',
 'not valid',
 'ounce',
 'oz',
 'pound',
 'quart',
 'tablespoon',
 'teaspoon'}